# Module 1.1 - introduction
## gas properties, heating value
This module is similar to the example module from Cantera: 
https://cantera.org/examples/jupyter/thermo/heating_value.ipynb.html
We are going to load a reaction mechanism and retrieve the heating values.
The complete reaction of methane and air is:

$$CH_4 + 2O_2 \rightarrow CO_2 + 2H_2O$$

We compute the lower heating value (LHV) as the difference in enthalpy (per kg mixture) between reactants and products at constant temperature and pressure, divided by the mass fraction of fuel in the reactants.

In [1]:
import cantera as ct
gas = ct.Solution('gri30.cti')

display the properties of the gas by simply calling the variable 

In [2]:
gas

In [3]:
gas()


  gri30:

       temperature             300  K
          pressure          101325  Pa
           density       0.0818891  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           26470        5.336e+04     J
   internal energy     -1.2109e+06       -2.441e+06     J
           entropy           64914        1.309e+05     J/K
    Gibbs function     -1.9448e+07        -3.92e+07     J
 heat capacity c_p           14312        2.885e+04     J/K
 heat capacity c_v           10187        2.054e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -15.7173
     [  +52 minor]              0                0



if you want to know more about the object, use help(object)

In [4]:
help(gas)

Help on Solution in module cantera.composite object:

class Solution(cantera._cantera.ThermoPhase, cantera._cantera.Kinetics, cantera._cantera.Transport)
 |  A class for chemically-reacting solutions. Instances can be created to
 |  represent any type of solution -- a mixture of gases, a liquid solution, or
 |  a solid solution, for example.
 |  
 |  Class `Solution` derives from classes `ThermoPhase`, `Kinetics`, and
 |  `Transport`.  It defines no methods of its own, and is provided so that a
 |  single object can be used to compute thermodynamic, kinetic, and transport
 |  properties of a solution.
 |  
 |  To skip initialization of the Transport object, pass the keyword argument
 |  ``transport_model=None`` to the `Solution` constructor.
 |  
 |  The most common way to instantiate `Solution` objects is by using a phase
 |  definition, species and reactions defined in an input file::
 |  
 |      gas = ct.Solution('gri30.cti')
 |  
 |  If an input file defines multiple phases, the p

the content of the python objects can be shown using dir()

In [5]:
dir(gas)

['DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'ID',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_references',
 'activities',
 'activity_coefficients',
 'add_reaction',
 'add_species',
 'atomic_weight',
 'atomic_weights',
 'basis',
 'binary_diff_coeffs',
 'chemical_potentials',
 'concentrations',
 'cp',
 'cp_mass',
 'cp_mole',
 'cr

ok, let's display the viscosity:

In [6]:
gas.viscosity

CanteraError: 
***********************************************************************
NotImplementedError thrown by Transport::viscosity:
Not implemented.
***********************************************************************


We cannot get the viscosity because we need to set a mixture and we need a model for the computation of mixture properties. Several models are available, from more accurate (and more expensive) to less accurate: **multi**, **mix**, **UnityLewis**
It is important to know which models are used because each model has it's own limitations and range of applicability.

How models and submodels are related for the transport class can be viewed here: 
https://cantera.org/documentation/docs-2.4/doxygen/html/d2/dfb/classCantera_1_1Transport.html

- **multi** multicomponent diffusion model https://www.cantera.org/documentation/docs-2.1/doxygen/html/classCantera_1_1MultiTransport.html
- **mix** mixture averaged model https://cantera.org/documentation/docs-2.4/doxygen/html/d9/d17/classCantera_1_1MixTransport.html
- **UnityLewis** unity lewis number https://cantera.org/documentation/docs-2.4/doxygen/html/d3/dd6/classCantera_1_1UnityLewisTransport.html

In [9]:
gas.transport_model = 'UnityLewis'
# Set reactants state
gas.TPX = 298, 101325, 'CH4:1, O2:2'

We have set a mixing model, so Cantera now can compute the mixture averaged properties like viscosity:

In [10]:
gas.viscosity

1.774771803936626e-05

In [11]:
rho = gas.density
k = gas.thermal_conductivity
cp = gas.cp_mass

we have the diffusion coefficients of each of the species available (why not a single value like viscosity?):

In [12]:
D = gas.mix_diff_coeffs_mass
print(D)

[2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05 2.26712245e-05 2.26712245e-05 2.26712245e-05
 2.26712245e-05]


The unity lewis number assumption means that $\frac{k}{c_p} = \rho\cdot D$

In [13]:
print(k/cp, rho*D[0])

2.4735970056741385e-05 2.4735970056741385e-05


In [14]:
h1 = gas.enthalpy_mass
Y_CH4 = gas['CH4'].Y[0] # returns an array, of which we only want the first element

# set state to complete combustion products without changing T or P
gas.TPX = None, None, 'CO2:1, H2O:2' 
h2 = gas.enthalpy_mass
LHV = -(h2-h1)/Y_CH4/1e6
print('LHV = {:.3f} MJ/kg'.format(LHV))

LHV = 50.026 MJ/kg


The LHV is calculated assuming that the water produced during combustion remains in the gas phase. However, more energy can be extracted from the mixture if this water is condensed, which is done in condensing boilers. This value is the higher heating value (HHV).

The ideal gas mixture model used here cannot calculate this contribution directly. However, Cantera also has a non-ideal equation of state which can be used to compute this contribution.

In [15]:
water = ct.Water()
# Set liquid water state, with vapor fraction x = 0
water.TX = 298, 0
h_liquid = water.h
# Set gaseous water state, with vapor fraction x = 1
water.TX = 298, 1
h_gas = water.h

# Calculate higher heating value
Y_H2O = gas['H2O'].Y[0]
HHV = -(h2-h1 + (h_liquid-h_gas) * Y_H2O)/Y_CH4/1e6
print('HHV = {:.3f} MJ/kg'.format(HHV))

HHV = 55.512 MJ/kg


The thermal efficiency of a heat exchanger is 100% if all heat can be extracted from the combustion products and the combustion products exit the heat exchanger at ambient temperature. Often, the lower heating value is used and the efficiency is 100% if we can extract 50.02 MJ from a kg of burnt methane. However, when the combustion products are cooled below the condensation temperature of water, we get an additional amount of heat, equal to the heat of evaporation. If the efficiency is still based on the heat retrieval compared to the chemical heat only, then the maximum efficiency is:

In [17]:
print('efficiency = {:.1f} %'.format(100.0*HHV/LHV))

efficiency = 111.0 %


This is the reason that condensing boilers sometimes claim to have more than 100% efficiency, because regulations sometimes demand that the lower heating value is used to determine the efficiency.

## Generalizing to arbitrary species

We can generalize this calculation by determining the composition of the products automatically rather than directly specifying the product composition. This can be done by computing the elemental mole fractions of the reactants mixture and noting that for complete combustion, all of the carbon ends up as $CO_2$, all of the hydrogen ends up as $H_2O$, and all of the nitrogen ends up as $N_2$. From this, we can compute the ratio of these species in the products.


In [18]:
# define a function that we can call with one argument and two return values
def heating_value(fuel):
    """ Returns the LHV and HHV for the specified fuel """
    gas.TP = 298, ct.one_atm
    # we set the equivalence ratio using the fuel
    gas.set_equivalence_ratio(1.0, fuel, 'O2:1.0')
    h1 = gas.enthalpy_mass
    Y_fuel = gas[fuel].Y[0]

    # complete combustion products, which are fuel dependent
    Y_products = {'CO2': gas.elemental_mole_fraction('C'),
                  'H2O': 0.5 * gas.elemental_mole_fraction('H'),
                  'N2': 0.5 * gas.elemental_mole_fraction('N')}

    gas.TPX = None, None, Y_products
    Y_H2O = gas['H2O'].Y[0]
    h2 = gas.enthalpy_mass
    LHV = -(h2-h1)/Y_fuel
    HHV = -(h2-h1 + (h_liquid-h_gas) * Y_H2O)/Y_fuel
    return LHV, HHV

fuels = ['H2', 'CH4', 'C2H6', 'C3H8', 'NH3', 'CH3OH']
print('fuel   LHV (MJ/kg)   HHV (MJ/kg)')
for fuel in fuels:
    LHV, HHV = heating_value(fuel)
    print('{:8s} {:7.3f}      {:7.3f}'.format(fuel, LHV/1e6, HHV/1e6))

fuel   LHV (MJ/kg)   HHV (MJ/kg)
H2       119.959      141.788
CH4       50.026       55.512
C2H6      47.511       51.901
C3H8      46.352       50.344
NH3       18.604       22.480
CH3OH     21.104       23.851


## Assignment 1.1:
- plot the viscosity of the methane-air mixture as a function of temperature for 3 different equivalence ratios: $\phi=[0.6,0.8,1.0]$
- determine the LHV and HHV in $\frac{MJ}{M^3}$. Make a bar graph comparing the mass and volumetric energy content of the above fuels. 